In [41]:
import pandas as pd
import numpy as np
import os
import env
from env import host, user, pwd
from pydataset import data
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import reg_wrangle

from sklearn.feature_selection import SelectKBest, RFE, f_regression, SequentialFeatureSelector

## 1.

In [12]:
tips = sns.load_dataset('tips')

In [13]:
tips

tips = tips.rename(columns={'size':'party_size'})

tips['gender_encoded'] = tips.sex.map({'Female': 1, 'Male': 0})

tips['smoker_encoded'] = tips.smoker.map({'No': 0, 'Yes': 1})

tips['time_encoded'] = tips.time.map({'Dinner': 1, 'Lunch': 0})

tips['thur'] = np.where(tips.day == 'Thur', 1, 0)
tips['fri'] = np.where(tips.day == 'Fri', 1, 0)
tips['sat'] = np.where(tips.day == 'Sat', 1, 0)
tips['sun'] = np.where(tips.day == 'Sun', 1, 0)

In [14]:
tips.head()

,total_bill,tip,sex,smoker,day,time,party_size,gender_encoded,smoker_encoded,time_encoded,thur,fri,sat,sun
0,16.99,1.01,Female,No,Sun,Dinner,2,1,0,1,0,0,0,1
1,10.34,1.66,Male,No,Sun,Dinner,3,0,0,1,0,0,0,1
2,21.01,3.50,Male,No,Sun,Dinner,3,0,0,1,0,0,0,1
3,23.68,3.31,Male,No,Sun,Dinner,2,0,0,1,0,0,0,1
4,24.59,3.61,Female,No,Sun,Dinner,4,1,0,1,0,0,0,1


## 1. a. 

In [15]:
# dot method calls .size messed up the column 
# need to use bracket method
tips['price_per_person'] = tips['total_bill'] / tips['party_size']

In [16]:
tips.head()

,total_bill,tip,sex,smoker,day,time,party_size,gender_encoded,smoker_encoded,time_encoded,thur,fri,sat,sun,price_per_person
0,16.99,1.01,Female,No,Sun,Dinner,2,1,0,1,0,0,0,1,8.495000
1,10.34,1.66,Male,No,Sun,Dinner,3,0,0,1,0,0,0,1,3.446667
2,21.01,3.50,Male,No,Sun,Dinner,3,0,0,1,0,0,0,1,7.003333
3,23.68,3.31,Male,No,Sun,Dinner,2,0,0,1,0,0,0,1,11.840000
4,24.59,3.61,Female,No,Sun,Dinner,4,1,0,1,0,0,0,1,6.147500


## 1. b. party size

In [17]:
# return the variables.   must prefix it with my wrangle file
train, validate, test = reg_wrangle.split_data(tips)

In [18]:
train.head()

,total_bill,tip,sex,smoker,day,time,party_size,gender_encoded,smoker_encoded,time_encoded,thur,fri,sat,sun,price_per_person
18,16.97,3.50,Female,No,Sun,Dinner,3,1,0,1,0,0,0,1,5.656667
172,7.25,5.15,Male,Yes,Sun,Dinner,2,0,1,1,0,0,0,1,3.625000
118,12.43,1.80,Female,No,Thur,Lunch,2,1,0,0,1,0,0,0,6.215000
28,21.70,4.30,Male,No,Sat,Dinner,2,0,0,1,0,0,1,0,10.850000
237,32.83,1.17,Male,Yes,Sat,Dinner,2,0,1,1,0,0,1,0,16.415000


In [19]:
X_train, y_train = train[['total_bill', 'price_per_person', 'gender_encoded', 'smoker_encoded', 'thur', 'fri','sat', 'sun', 'time_encoded', 'party_size']], train.tip
X_validate, y_validate = validate[['total_bill', 'price_per_person', 'gender_encoded', 'smoker_encoded', 'thur', 'fri','sat', 'sun', 'time_encoded', 'party_size']], validate.tip
X_test, y_test = test[['total_bill', 'price_per_person', 'gender_encoded', 'smoker_encoded', 'thur', 'fri','sat', 'sun', 'time_encoded', 'party_size']], test.tip

In [20]:
# make the thing    k= 2 is picking the top 2 features
kbest = SelectKBest(f_regression, k=2)
# fit the thing
_ = kbest.fit(X_train, y_train)

In [21]:
# statistical f-value:
kbest.scores_
#p value: 
kbest.pvalues_

array([7.18647033e-20, 1.31032690e-03, 2.84479443e-01, 5.57997755e-01,
       3.01377404e-01, 4.06823770e-01, 9.55046793e-01, 1.23662519e-01,
       1.82144884e-01, 1.34164190e-12])

In [22]:
kbest_results = pd.DataFrame(
    dict(p=kbest.pvalues_, f=kbest.scores_),
                             index = X_train.columns)

In [23]:
kbest_results

,p,f
total_bill,7.186470e-20,115.984909
price_per_person,1.310327e-03,10.777792
gender_encoded,2.844794e-01,1.154792
smoker_encoded,5.579978e-01,0.344909
thur,3.013774e-01,1.076380
fri,4.068238e-01,0.692418
sat,9.550468e-01,0.003190
sun,1.236625e-01,2.400404
time_encoded,1.821449e-01,1.798647
party_size,1.341642e-12,61.259089


## 1. c.

In [24]:
# get-support() will output a boolean mask to tell me which features were selected
# we can apply this mask to the columns in our original dataframe
X_train.columns[kbest.get_support()]

Index(['total_bill', 'party_size'], dtype='object')

In [25]:
# kbest transform will convert our information to the selected feature subspace
# its a numpy array
kbest.transform(X_train)[:5]

array([[16.97,  3.  ],
       [ 7.25,  2.  ],
       [12.43,  2.  ],
       [21.7 ,  2.  ],
       [32.83,  2.  ]])

In [26]:
X_train_transformed = pd.DataFrame(
    kbest.transform(X_train),
    columns=X_train.columns[kbest.get_support()],
    index=X_train.index
)

In [27]:
X_train_transformed.head(10)

,total_bill,party_size
18,16.97,3.0
172,7.25,2.0
118,12.43,2.0
28,21.70,2.0
237,32.83,2.0
207,38.73,4.0
183,23.17,4.0
60,20.29,2.0
41,17.46,2.0
160,21.50,4.0


## 1. d.

In [28]:
from sklearn.linear_model import LinearRegression

In [29]:
# make a model object to use in RFE process.
# The model is here to give us metrics on feature importance and model score
# allowing us to recursively reduce the number of features to reach our desired space
model = LinearRegression()

In [30]:
# make thing
rfe = RFE(model, n_features_to_select=2)
# fit thing
rfe.fit(X_train, y_train)

RFE(estimator=LinearRegression(), n_features_to_select=2)

In [31]:
rfe.ranking_

array([5, 3, 2, 7, 1, 6, 4, 8, 9, 1])

In [32]:
pd.DataFrame(
{
    'rfe_ranking': rfe.ranking_
},index = X_train.columns)

,rfe_ranking
total_bill,5
price_per_person,3
gender_encoded,2
smoker_encoded,7
thur,1
fri,6
sat,4
sun,8
time_encoded,9
party_size,1


In [33]:
rfe.get_support()

array([False, False, False, False,  True, False, False, False, False,
        True])

In [34]:
X_train_transformed = pd.DataFrame(
rfe.transform(X_train),
index = X_train.index,
columns = X_train.columns[rfe.support_])

In [35]:
X_train_transformed.head()

,thur,party_size
18,0.0,3.0
172,0.0,2.0
118,1.0,2.0
28,0.0,2.0
237,0.0,2.0


## 2.

In [36]:
def select_kbest(X_train, y_train):
    kbest = SelectKBest(f_regression, k=3)
    _ = kbest.fit(X_train, y_train)
    kbest_results = pd.DataFrame(
        dict(p=kbest.pvalues_, f=kbest.scores_),
                                 index = X_train.columns)
    return kbest_results

## 3. 

In [38]:
def rfe(model, n_features_to_select=2):
    model = LinearRegression()
    rfe = RFE(model, n_features_to_select=2)
    rfe.fit(X_train, y_train)
    rfe.ranking_
    pd.DataFrame(
    {
        'rfe_ranking': rfe.ranking_
    },index = X_train.columns)
    rfe.get_support()
    X_train_transformed = pd.DataFrame(
    rfe.transform(X_train),
    index = X_train.index,
    columns = X_train.columns[rfe.support_])
    return X_train_transformed

In [39]:
rfe(model, n_features_to_select=2)

,thur,party_size
18,0.0,3.0
172,0.0,2.0
118,1.0,2.0
28,0.0,2.0
237,0.0,2.0
207,0.0,4.0
183,0.0,4.0
60,0.0,2.0
41,0.0,2.0
160,0.0,4.0


## 4. 

In [43]:
swiss = data('swiss')

In [44]:
swiss.head()

,Fertility,Agriculture,Examination,Education,Catholic,Infant.Mortality
Courtelary,80.2,17.0,15,12,9.96,22.2
Delemont,83.1,45.1,6,9,84.84,22.2
Franches-Mnt,92.5,39.7,5,5,93.40,20.2
Moutier,85.8,36.5,12,7,33.77,20.3
Neuveville,76.9,43.5,17,15,5.16,20.6


In [45]:
# return the variables.   must prefix it with my wrangle file
train, validate, test = reg_wrangle.split_data(swiss)

In [46]:
train.head()

,Fertility,Agriculture,Examination,Education,Catholic,Infant.Mortality
Rolle,60.5,60.8,16,10,7.72,16.3
Lavaux,65.1,73.0,19,9,2.84,20.0
Nyone,56.6,50.9,22,12,15.14,16.7
Conthey,75.5,85.9,3,2,99.71,15.1
Yverdon,65.4,49.5,15,8,6.10,22.5


In [47]:
X_train, y_train = train[['Agriculture', 'Examination', 'Education', 'Catholic', 'Infant.Mortality']], train.Fertility

In [48]:
select_kbest(X_train, y_train)

,p,f
Agriculture,0.399514,0.736900
Examination,0.006369,9.008418
Education,0.191454,1.811436
Catholic,0.000355,17.518694
Infant.Mortality,0.024663,5.779868


In [50]:
# you can make another function that doesnt stop at the above df
# which as you can see the higher f score and lower the pvalue
# are gonna be the best
#-----   X_train.columns[kbest.get_support()]

In [60]:
def select_kbest(X_train, y_train):
    kbest = SelectKBest(f_regression, k=2)
    _ = kbest.fit(X_train, y_train)
    kbest_results = pd.DataFrame(
        dict(p=kbest.pvalues_, f=kbest.scores_),
                                 index = X_train.columns)
    return kbest_results

In [61]:
X_train, y_train = train[['Agriculture', 'Examination', 'Education', 'Catholic', 'Infant.Mortality']], train.Fertility

In [62]:
select_kbest(X_train, y_train)

,p,f
Agriculture,0.399514,0.736900
Examination,0.006369,9.008418
Education,0.191454,1.811436
Catholic,0.000355,17.518694
Infant.Mortality,0.024663,5.779868
